<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Practicing Web Scraping With XPath

_Authors: Dave Yerrington (SF)_

---

### Learning Objectives
*After this lesson, you will be able to:*
- Build a basic scraper
- Understand HTML and XPath basics
- Scrape a website for various data and putting your results into a DataFrame

### Lesson Guide
- [Review of HTML and Web Scraping](#review1)
- [Review of XPath](#review2)
- [Basic XPath Expressions](#basic-xpath)
    - [Absolute References](#absolute)
    - [Relative References](#relative-references)
    - [Selecting Attributes](#attributes)
- [Guided Practice: Where's Waldo? - XPath Edition](#practice1)
- [1 vs. N Selections](#1vsn)
    - [Selecting the First Element in a Series of Elements](#first-elem)
    - [Selecting the Last Element in a Series of Elements](#last-elem)
    - [Selecting all Elements Matching a Selection](#all-elem-match)
    - [Selecting Elements Matching an Attribute](#elem-match-attr)
- [Guided Practice: Selecting Elements](#practice2)
- [A Quick Note: The Requests Module](#requests)
- [Guided Practice: Scrape DataTau Headlines](#practice3)
- [Independent Practice](#independent)

<a id='review1'></a>
## Review of HTML and Web Scraping

---

Web scraping is a technique of extracting information from websites. It's the downloading and transformation of unstructured web data into structured data that can be stored and analyzed.

There are a variety of ways to "scrape" what we want from the web:
- Using third-party services (import.io).
- By writing our own Python apps that pull HTML documents and parse them.
  - Mechanize
  - Scrapy
  - Requests
  - Libxml/XPath
  - Regular expressions
  - BeautifulSoup

> **Check:** What do you perceive to be the hardest aspect of scraping?

_E.G.: If you were asked to scrape Craigslist property listings and put them in a DataFrame, what would hold you up?_

### Review: HTML

In the HTML document object model (DOM), everything is a node:
 * The document itself is a document node
 * All HTML elements are element nodes
 * All HTML attributes are attribute nodes
 * The text inside HTML elements are text nodes
 * The comments are comment nodes

### Review: Elements
Elements begin and end with **opening and closing tags** that are defined by namespaced, encapsulated strings. 

```html
<title>I am a title.</title>
<p>I am a paragraph.</p>
<strong>I am bold.</strong>
```

_Note: The tags **title, p,** and **strong** are represented below._

### Review: Element Parent/Child Relationships

<img src="http://www.htmlgoodies.com/img/2007/06/flowChart2.gif" width="250">

**Elements begin and end in the same namespace, like so:** `<p></p>`

**Elements can have parents and children.** It's important to remember that an element can be both a parent and a child, and whether to refer to the element as a parent or a child depends on the specific element you are referencing.

_Your parents are **parents** to you but **children** of your grandparents. The same logic applies with HTML elements._

```html
<body id = 'parent'>
    <div id = 'child_1'>I am the child of 'parent.'
        <div id = 'child_2'>I am the child of 'child_1.'
            <div id = 'child_3'>I am the child of 'child_2.'
                <div id = 'child_4'>I am the child of 'child_4.'</div>
            </div>
        </div>
    </div>
</body>
```

**or**

```html
<body id = 'parent'>
    <div id = 'child_1'>I am the parent of 'child_2.'
        <div id = 'child_2'>I am the parent of 'child_3.'
            <div id = 'child_3'> I am the parent of 'child_4.'
                <div id = 'child_4'>I am not a parent. </div>
            </div>
        </div>
    </div>
</body>
```

### Review: Element Attributes

Elements can also have attributes. Attributes are defined inside **element tags** and can contain data that may be useful to scrape.

```html
<a href="http://lmgtfy.com/?q=html+element+attributes" title="A title" id="web-link" name="hal">A Simple Link</a>
```

The **element attributes** of this `<a>` tag element are:
- `id`
- `href`
- `title`
- `name`

This `<a>` tag example will render in your browser like this:
> <a href="https://www.youtube.com/watch?v=dQw4w9WgXcQ">A Simple Link</a>


**Check:** Can you identify an attribute, an element, a text item, and a child element in the code below?

```HTML
<html>
   <title id="main-title">All this scraping is making me itch!</title>
   <body>
       <h1>Welcome to my Homepage</h1>
       <p id="welcome-paragraph" class="strong-paragraph">
           <span>Hello friends, let me tell you about this cool hair product.</span>
           <ul>
              <li>It's cool.</li>
              <li>It's fresh.</li>
              <li>It can tell the future.</li>
              <li>Always be closing.</li>
           </ul>
       </p>
   </body>
```

**Bonus:** What's missing?

In [1]:
# </html>

<a id='review2'></a>
## Review of XPath

---

XPath uses path expressions to select nodes or node sets in an HTML/XML document. These path expressions look similar to the expressions you see when you work with a traditional computer file system.

### XPath Features

XPath includes more than 100 built-in functions to help us select and manipulate HTML (or XML) documents. XPath has functions for:

- String values
- Numeric values
- Date and time comparison
- Sequence manipulation
- Boolean values
- And more

<a id='basic-xpath'></a>
## Basic XPath Expressions

---

XPath comes with a wide array of features, but the basics of selecting data are the most common problems that XPath can help you solve.

Most often, you'll use **XPath** for selecting data from HTML documents. There are two ways you can **select elements** within HTML using **XPath**:

- Absolute references
- Relative references

<a id='absolute'></a>
### Absolute References

> _For our XPath demonstration, we'll use Scrapy, which is using [Libxml](http://xmlsoft.org) under the hood. Libxml provides the basic functionality for XPath expressions._

In [2]:
# Pip install Scrapy.
# Pip install --upgrade zope2.
from scrapy.selector import Selector
from scrapy.http import HtmlResponse

HTML = """
<html>
    <body>
        <span id="only-span">good</span>
    </body>
</html>
"""
# The same thing, but an absolute reference:
Selector(text=HTML).xpath('/html/body/span/text()').extract()


['good']

<a id='relative-references'></a>
### Relative References

Relative references in XPath match the "ends" of structures. As there is only a single `span` element, `//span/text()` matches **one element**.

In [3]:
Selector(text=HTML).xpath('//span/text()').extract()

['good']

<a id='attributes'></a>
### Selecting Attributes

Attributes are **within a tag**, such as `id="only-span"` within our `span` attribute. We can get the attribute by using the `@` symbol **after** the **element reference**.


In [4]:
Selector(text=HTML).xpath('//span/@id').extract()

['only-span']

<a id='practice1'></a>
## Guided Practice: Where's Waldo? — XPath Edition

---

**In this example, we'll find Waldo together. Find Waldo as:**

- An element
- An attribute
- A text element

The practice HTML string is provided below.

In [5]:
HTML = """
<html>
    <body>
        
        <ul id="waldo">
            <li class="waldo">
                <span> yo I'm not here</span>
            </li>
            <li class="waldo">Height:  ???</li>
            <li class="waldo">Weight:  ???</li>
            <li class="waldo">Last Location:  ???</li>
            <li class="nerds">
                <div class="alpha">Bill Gates</div>
                <div class="alpha">Zuckerberg</div>
                <div class="beta">Theil</div>
                <div class="animal">Parker</div>
            </li>
        </ul>
        
        <ul id="tim">
            <li class="tdawg">
                <span>yo im here</span>
            </li>
        </ul>
        <li>stuff</li>
        <li>stuff2</li>
        
        <div id="cooldiv">
            <span class="dsi-rocks">
               YO!
            </span>
        </div>
        
        
        <waldo>Waldo</waldo>
    </body>
</html>
"""

In [6]:
Selector(text=HTML).xpath('//@class').extract()

['waldo',
 'waldo',
 'waldo',
 'waldo',
 'nerds',
 'alpha',
 'alpha',
 'beta',
 'animal',
 'tdawg',
 'dsi-rocks']

In [10]:
# Find the absolute element.
print(Selector(text=HTML).xpath('/html/body/waldo/text()').extract())
print(Selector(text=HTML).xpath('/html/body/ul/li/text()').extract())

# Find the relative element.
print(Selector(text=HTML).xpath('//li').extract())

# Find the element attribute.
print(Selector(text=HTML).xpath('////@class').extract())
print(Selector(text=HTML).xpath('//ul/@id').extract())


['Waldo']
['\n                ', '\n            ', 'Height:  ???', 'Weight:  ???', 'Last Location:  ???', '\n                ', '\n                ', '\n                ', '\n                ', '\n            ', '\n                ', '\n            ']
['<li class="waldo">\n                <span> yo I\'m not here</span>\n            </li>', '<li class="waldo">Height:  ???</li>', '<li class="waldo">Weight:  ???</li>', '<li class="waldo">Last Location:  ???</li>', '<li class="nerds">\n                <div class="alpha">Bill Gates</div>\n                <div class="alpha">Zuckerberg</div>\n                <div class="beta">Theil</div>\n                <div class="animal">Parker</div>\n            </li>', '<li class="tdawg">\n                <span>yo im here</span>\n            </li>', '<li>stuff</li>', '<li>stuff2</li>']
['waldo', 'waldo', 'waldo', 'waldo', 'nerds', 'alpha', 'alpha', 'beta', 'animal', 'tdawg', 'dsi-rocks']
['waldo', 'tim']


In [11]:
print(Selector(text=HTML).xpath('//@class').extract())

['waldo', 'waldo', 'waldo', 'waldo', 'nerds', 'alpha', 'alpha', 'beta', 'animal', 'tdawg', 'dsi-rocks']


In [12]:
print(Selector(text=HTML).xpath('//*/@class').extract())

['waldo', 'waldo', 'waldo', 'waldo', 'nerds', 'alpha', 'alpha', 'beta', 'animal', 'tdawg', 'dsi-rocks']


<a id='1vsn'></a>
## 1 vs N Selections

---

When selecting elements via relative reference, it's possible that you'll select multiple items. It's still possible to select single items if you're specific enough.

**Singular Reference**
- **Index** starts at **1**
- Selections by offset
- Selections by "first" or "last"
- Selections by **unique attribute value**


In [13]:
HTML = """
<html>
    <body>
    
        <!-- Search Results -->
        <div class="search-result">
           <a href="https://www.youtube.com/watch?v=751hUX_q0Do" title="Rappin with Gas">Rapping with gas</a>
           <span class="link-details">This is a great video about gas.</span>
        </div>
        <div class="search-result">
           <a href="https://www.youtube.com/watch?v=97byWqi-zsI" title="Casio Rapmap">The Rapmaster</a>
           <span class="link-details">My first synth ever.</span>
        </div>
        <div class="search-result">
           <a href="https://www.youtube.com/watch?v=TSwqnR327fk" title="Cinco Products">Cinco Midi Organizer</a>
           <span class="link-details">Midi files at the speed of light.</span>
        </div>
        <div class="search-result">
           <a href="https://www.youtube.com/watch?v=8TCxE0bWQeQ" title="Baddest Gates">BBG Baddest Moments</a>
           <span class="link-details">It's tough to be a gangster.</span>
        </div>
        
        <!-- Page stats -->
        <div class="page-stats-container">
            <li class="item" id="pageviews">1,333,443</li>
            <li class="item" id="somethingelse">bla</li>
            <li class="item" id="last-viewed">01-22-2016</li>
            <li class="item" id="views-per-hour">1,532</li>
            <li class="item" id="kiefer-views-per-hour">5,233.42</li>
        </div>
        
    </body>
</html>
"""

span = Selector(text=HTML).xpath('/html/body/div/li[@id="kiefer-views-per-hour"]/text()').extract()
span

['5,233.42']

<a id='first-elem'></a>
### Selecting the First Element in a Series of Elements

In [14]:
spans = Selector(text=HTML).xpath('//span').extract()
spans[0]

'<span class="link-details">This is a great video about gas.</span>'

<a id='last-elem'></a>
### Selecting the Last Element in a Series of Elements

In [15]:
spans = Selector(text=HTML).xpath('//span').extract()
spans[-1]

'<span class="link-details">It\'s tough to be a gangster.</span>'

<a id='all-elem-match'></a>
### Selecting All Elements Matching a Selection

In [16]:
Selector(text=HTML).xpath('//span').extract()

['<span class="link-details">This is a great video about gas.</span>',
 '<span class="link-details">My first synth ever.</span>',
 '<span class="link-details">Midi files at the speed of light.</span>',
 '<span class="link-details">It\'s tough to be a gangster.</span>']

<a id='elem-match-attr'></a>
### Selecting Elements Matching An _Attribute_

This will be one of the most common ways you'll select items. HTML DOM elements will be more differentiated based on their class and ID variables. Mainly, these types of attributes are used by web developers to refer to specific elements or a broad set of elements to apply visual characteristics to using CSS.

```HTML 
//element[@attribute="value"]
```

**Generally:**

- "Class" attributes within elements usually refer to multiple items
- "ID" attributes are supposed to be unique but aren't always

_CSS stands for cascading style sheets. These are used to abstract the definition of visual elements on a micro and macro scale for the web. They are also our best friend as data miners. They give us strong hints and cues as to how a web document is structured._

<a id='practice2'></a>
## Guided Practice: Selecting Elements

---

1) How can we get a series of only text items for the page statistics section of our page?<br>
2) We want to know only how many times Kiefer views the YouTube videos page per hour.

In [17]:
# Get all text elements for the page statistics section.
Selector(text=HTML).xpath('//li/text()')

[<Selector xpath='//li/text()' data='1,333,443'>,
 <Selector xpath='//li/text()' data='bla'>,
 <Selector xpath='//li/text()' data='01-22-2016'>,
 <Selector xpath='//li/text()' data='1,532'>,
 <Selector xpath='//li/text()' data='5,233.42'>]

In [18]:
# Get only the text for "Kiefer's" number of views per hour.
Selector(text=HTML).xpath('//div[@class="page-stats-container"]/li[4]/text()').extract()

['1,532']

In [19]:
# Get only the text for "Kiefer's" number of views per hour.
Selector(text=HTML).xpath('//li[@id="kiefer-views-per-hour"]/text()').extract()

['5,233.42']

<a id='requests'></a>
## A Quick Note on the Requests Module

---

The requests module is the gateway to interacting with the web using Python. We can:

 - Fetch web documents as strings
 - Decode JSON
 - Complete basic data munging with web documents
 - Download static files that aren't text, including:
  - Images
  - Videos
  - Binary data


Take some time and read up on requests:

http://docs.python-requests.org/en/master/user/quickstart/

<a id='practice3'></a>
## Guided Practice: Scrape DataTau Headlines

DataTau is a great site for data science news. Let's take its headlines using Python **`requests`** and practice selecting various elements.

Using the <a href="https://chrome.google.com/webstore/detail/xpath-helper/hgimnogjllphhhkhlmebbmlgjoejdpjl?hl=en">XPath Helper Chrome plugin</a> _(cmd-shift-x)_ and the Chrome Inspect feature, let's explore the structure of the page.

_Here's a <a href="https://www.youtube.com/watch?v=i2Li1vnv09U">concise video</a> that demonstrates the basic Inspect feature in Chrome._

In [20]:
# Please only run this frame once to avoid hitting the site too hard all at once.
import requests

response = requests.get("http://www.datatau.com")
HTML = response.text  
HTML[0:150]           # View the first 500 characters of the HTML index document for DataTau.

'<html><head><link rel="stylesheet" type="text/css" href="news.css">\n<link rel="shortcut icon" href="http://www.iconj.com/ico/d/x/dxo02ap56v.ico">\n<scr'

### Selecting Only the Headlines

We'll use the XPath Helper tool to inspect the markup that comprises the **title** to find a pattern. Because there is more than one **title**, we expect to find a series of elements representing the **title** data we're interested in.

In this example, we are referencing the **first center**, **third table row (`tr[3]`)** within the second **td having a class of "title" (`td[@class="title"][2]`)** and the anchor tag within a **(`a/text()`)**.


In [21]:
import pandas as pd

titles = Selector(text=HTML).xpath('//td[@class="title"]/a/text()').extract()
titles[0:10] # The first five titles.

['Comparison of the Most Useful Text Processing APIs',
 'How to deliver on Machine Learning projects',
 'Analyzing news headline sentiment with Python',
 'Five Key Roles in Data Product Management',
 'Insight Data Engineering & DevOps Engineering land in Seattle',
 'Polyaxon 0.2.3: Experiment stability, better tracking, better visualization',
 'Calculating the Age of the Universe',
 'A/B Testing with a Data Science Approach',
 '10 Google Analytics reports to grow your blog and content',
 'Top Data Science Interview Questions and Answers - Acing the AI Interview']

### How Can We Get the URLs From the Titles?

In [22]:
urls = Selector(text=HTML).xpath('//td[@class="title"]/a/@href').extract()
urls[::-1]

['/x?fnid=ko1F4BAu06',
 'https://semanti.ca/blog/?the-most-important-skills-for-a-data-scientist',
 'https://theexecute.com/automate-blog-posts-subscribers/',
 'https://www.gethighlights.co/blog/fix-analytics-dashboards/',
 'http://www.behind-the-enemy-lines.com/2018/01/how-many-mechanical-turk-workers-are.html',
 'https://medium.com/acing-ai/ibm-artificial-intelligence-interview-questions-57ce1de3f6cf',
 'https://data36.com/productive-data-scientist-not-more-smarter/',
 'https://medium.com/activewizards-machine-learning-company/u-s-startups-locations-analysis-ab1e9cbf38df',
 'https://data36.com/sql-data-analysis-workshop-online/',
 'https://www.kettlebellkings.com/blog/kettlebells-muscular-endurance-for-combat-athletes/',
 'https://medium.com/indeed-data-science/where-do-data-scientists-come-from-fc526023ace',
 'https://github.com/polyaxon/rhea',
 'https://www.indicative.com/blog/real-time-data-analysis-efficient-business/',
 'https://medium.com/feature-labs-engineering/featuretools-o

In [23]:
titles[0:5] # The first five titles.

['Comparison of the Most Useful Text Processing APIs',
 'How to deliver on Machine Learning projects',
 'Analyzing news headline sentiment with Python',
 'Five Key Roles in Data Product Management',
 'Insight Data Engineering & DevOps Engineering land in Seattle']

### How Can We Get the Site Domain After the Title Within the Parentheses (i.e., stitchfix.com)?

In [24]:
domains = Selector(text=HTML).xpath("//span[@class='comhead']/text()").extract()

In [25]:
domains[0:5]

[' (medium.com) ',
 ' (insightdatascience.com) ',
 ' (interviewqs.com) ',
 ' (insightdatascience.com) ',
 ' (insightdatascience.com) ']

### How About the Points?

In [26]:
points = Selector(text=HTML).xpath('//td[@class="subtext"]/span/text()').extract()
points[0:5]

['8 points', '6 points', '3 points', '5 points', '2 points']

### How About the "More" Link?

> *Hint: You can use `element[text()='exact text']` to find text elements matching specific text.*

In [27]:
next_link = Selector(text=HTML).xpath('//a[text()="More"]/@href').extract()
next_link

['/x?fnid=ko1F4BAu06']

<a id='independent'></a>
## Independent Practice

---

**For the next 30 minutes, try to grab the following from DataTau:**

- Story titles
- Story URL (href)
- Domain
- Points

**Stretch goals:**
- Author
- Comment count

**Put your results into a DataFrame.**

- Perform a basic analysis of domains and point distributions

**Bonus**

Automatically find the next "more" link and mine the next page(s) until none exist. Logically, you can parse each page with this pseudocode:

- Does the next link exist (a tag with `text == "More"`)?
- Fetch the URL, prepended with domain (`datatau.com/(extracted link here)`)
- Parse the page with `Selector(text=HTML).xpath('').extract()` to find the elements
- Add to DataFrame

> _Note: You might want to set a limit — something like 2–3 total requests per attempt — to avoid unnecessary transfer._


In [28]:
url="http://www.datatau.com"
response  =  requests.get(url)
links     =  Selector(text=response.text).xpath("//td[@class='title']/a/@href").extract()
titles    =  Selector(text=response.text).xpath("//td[@class='title']/a/text()").extract()
points    =  Selector(text=response.text).xpath("//td[@class='subtext']/span/text()").extract()
domains   =  Selector(text=response.text).xpath("//td[@class='title']/span/text()").extract()
authors   =  Selector(text=response.text).xpath("//td[@class='subtext']/a[contains(@href, 'user')]/text()").extract()
comments  =  Selector(text=response.text).xpath("//td[@class='subtext']/a[contains(@href, 'item')]/text()").extract()

expected_length = 30

# Add [np.nan]*(expected_length - len(points)) to the end of the lists. It will fill in missing
# values that sometimes don't exist at the end of the results.
# scraped = dict(
#     titles   =  titles[:30], 
#     links    =  links[:30], # :30 because of that "more" link.
#     points   =  points + [np.nan]*(expected_length - len(points)),
#     domains  =  domains + [np.nan]*(expected_length - len(domains)),
#     authors  =  authors + [np.nan]*(expected_length - len(authors)),
#     comments =  comments + [np.nan]*(expected_length - len(comments))
# )

In [29]:
titles

['Comparison of the Most Useful Text Processing APIs',
 'How to deliver on Machine Learning projects',
 'Analyzing news headline sentiment with Python',
 'Five Key Roles in Data Product Management',
 'Insight Data Engineering & DevOps Engineering land in Seattle',
 'Polyaxon 0.2.3: Experiment stability, better tracking, better visualization',
 'Calculating the Age of the Universe',
 'A/B Testing with a Data Science Approach',
 '10 Google Analytics reports to grow your blog and content',
 'Top Data Science Interview Questions and Answers - Acing the AI Interview',
 'MapD has rebranded to OmniSci',
 'A Machine Learning Cheatsheet',
 "Introduction to time series forecasting in Python with FB's Prophet",
 'ECCV 2018 conference recap',
 'Introduction to KNN in R',
 'A Brief History of Machine Learning Models Explainability',
 'Working with Data Feeds',
 'Distributed feature engineering with Spark',
 'How Real-Time Data Can Make Your Business More Efficient',
 'Rhea: Efficient environment va

In [30]:
len(titles),len(links),len(authors)

(31, 31, 30)

In [32]:
import requests, numpy as np

def parse_url(url="http://www.datatau.com", data=False):
    
    response  =  requests.get(url)
    links     =  Selector(text=response.text).xpath("//td[@class='title']/a/@href").extract()
    titles    =  Selector(text=response.text).xpath("//td[@class='title']/a/text()").extract()
    points    =  Selector(text=response.text).xpath("//td[@class='subtext']/span/text()").extract()
    domains   =  Selector(text=response.text).xpath("//td[@class='title']/span/text()").extract()
    authors   =  Selector(text=response.text).xpath("//td[@class='subtext']/a[contains(@href, 'user')]/text()").extract()
    comments  =  Selector(text=response.text).xpath("//td[@class='subtext']/a[contains(@href, 'item')]/text()").extract()

    expected_length = 30
    
    # Add [np.nan]*(expected_length - len(points)) to 
    # the end of the lists. It will fill in missing
    # values that sometimes don't exist at the end of the results.
    scraped = dict(
        titles   =  titles[:30], 
        links    =  links[:30], # :30 because of that "more" link.
        points   =  points + [np.nan]*(expected_length - len(points)),
        domains  =  domains + [np.nan]*(expected_length - len(domains)),
        authors  =  authors + [np.nan]*(expected_length - len(authors)),
        comments =  comments + [np.nan]*(expected_length - len(comments))
    )
    
    df = pd.DataFrame(scraped)
    
    if type(data) != bool:
        data = df.append(data)
    else:
        data = df
        
    # If there's data, append them. 
    # If not, it's the first iteration so there's no need.
    # Find "more" link.
    more_anchor  =  Selector(text=response.text).xpath("//a[text() = 'More']/@href").extract()
    
    if len(more_anchor) > 0:
        more_url  =  "http://www.datatau.com%s" % more_anchor[0]
        print("Fetching %s..." % more_url)
        return parse_url(more_url, data=data)
    else:
        return data.reset_index()
       
        
df = parse_url("http://www.datatau.com")
df

Fetching http://www.datatau.com/x?fnid=ko1F4BAu06...
Fetching http://www.datatau.com/x?fnid=GDeIRwlKNn...
Fetching http://www.datatau.com/x?fnid=GZSRkB3kTQ...
Fetching http://www.datatau.com/x?fnid=9EeRUaFvpy...
Fetching http://www.datatau.com/x?fnid=3EkcOsVH9T...
Fetching http://www.datatau.com/x?fnid=bSyBYTe85G...


,index,titles,links,points,domains,authors,comments
0,0,Webinar on Survey Analysis using Machine Learning,https://blog.paralleldots.com/product/webinar-...,4 points,(paralleldots.com),shashankg22,discuss
1,1,Create TensorFlow Name Scopes For TensorBoard,https://www.aiworkbox.com/lessons/create-tenso...,3 points,(aiworkbox.com),iheartai,discuss
2,2,AI and Data Science trends (ahead of DataWorks...,https://medium.com/inside-machine-learning/ai-...,3 points,(medium.com),m_ly,discuss
3,3,How to get a free Lifeline Phone service in Maine,https://www.tagmobile.com/blog/how-to-get-a-fr...,3 points,(tagmobile.com),tagmobile56007,discuss
4,4,Hadoop 3: Comparison with Hadoop 2 and Spark,https://medium.com/activewizards-machine-learn...,25 points,(medium.com),tbugaevskii,discuss
5,5,Python Import Statement & Built-in Modules for...,https://data36.com/python-import-built-in-modu...,16 points,(data36.com),hiimtomi,discuss
6,6,Elitist shuffle for recommendation systems,https://blog.godatadriven.com/rod-elitist-shuffle,15 points,(godatadriven.com),stijntonk,discuss
7,7,"Getting Started with IPFS, Python, and Jupyter...",https://hackernoon.com/a-short-trip-to-jupyter...,3 points,(hackernoon.com),andrewxhill,discuss
8,8,Reading and writing to Google Spreadsheets usi...,https://www.interviewqs.com/blog/python_sheets,5 points,(interviewqs.com),ddinterview,discuss
9,9,"An Interview with Jesse Steinweg-Woods — Ph.D,...",https://medium.com/acing-ai/in-conversation-wi...,3 points,(medium.com),vimarshk,discuss
